# 特征工程

1. 实现目标需要哪些数据，这些数据能否获得，覆盖率，准确率
2. 如何获得数据，如何保存
3. 特征处理
   1. 特征清洗：清洗异常样本，采样（数据不平衡/样本权重）
   2. 预处理
      1. 单个特征：归一化/离散化/dummy coding/缺失值/数据变换
      2. 多个特征：降维（PCA/LDA）,特征选择（Filter/Wrapper/Embedded）
      3. 衍生变量
4. 特征监控
   1. 特征有效性分析
   2. 监控重要特征



In [2]:
# 加载鸢尾花数据
from sklearn.datasets import load_iris
iris = load_iris()
iris.data.shape,iris.target.shape

((150, 4), (150,))

In [ ]:
# 数据预处理

## 无量纲化
### 标准化
from sklearn.preprocessing import StandardScaler
StandardScaler().fit_transform(iris.data)
### 区间缩放法
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(iris.data)
### 归一化 ,标准化是依照特征矩阵的列处理数据，通过z-score，归一化是一种特征矩阵行处理数据，在计算时有统一的标准，都转化为单位向量
from sklearn.preprocessing import Normalizer
Normalizer().fit_transform(iris.data)

## 对定量特征二值化
from sklearn.preprocessing import Binarizer
Binarizer(threshold=3).fit_transform(iris.data)# 阈值为3分为0，1

## 对定性特征哑编码
from sklearn.preprocessing import OneHotEncoder
OneHotEncoder().fit_transform(iris.target.reshape((-1,1)))

## 缺失值计算
#缺失值计算，返回值为计算缺失值后的数据
#参数missing_value为缺失值的表示形式，默认为NaN
#参数strategy为缺失值填充方式，默认为mean（均值）
from numpy import vstack,array,nan
from sklearn.preprocessing import Imputer
Imputer().fit_transform(iris.data)

## 数据变换
### 多项式变换
from sklearn.preprocessing import PolynomialFeatures
PolynomialFeatures().fit_transform(iris.data)
### 对数变换
from numpy import log1p
from sklearn.preprocessing import FunctionTransformer
FunctionTransformer(log1p).fit_transform(iris.data)


### 特征选择

1. 两方面选择
   1. 特征是否发散，需要发散才有意义
   2. 特征与目标的相关性
2. 方法
   1. Filter 过滤法，按照发散性和相关性对特征评分，设定阈值后选择
   2. Wrapper 包装发，根据目标函数，即预测效果评分，每次选择若干或者排除若干特征
   3. Embedded 集成法，使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，从大到小选择特征，类似于Filter，但是通过训练确定特征优劣。


In [ ]:
# 特征选择

## Filter
### 方差选择法
from sklearn.feature_selection import VarianceThreshold
VarianceThreshold(threshold=3).fit_transform(iris.data)
### 相关系数选择法
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
#选择K个最好的特征，返回选择特征后的数据
#第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
#参数k为选择的特征个数
SelectKBest(lambda X,Y:array(map(lambda x:pearsonr(x,Y),X.T)).T,k=2).fit_transform(iris.data,iris.target)
### 互信息法
from sklearn.feature_selection import SelectKBest
from minepy import MINE
#由于MINE的设计不是函数式的，定义mic方法将其为函数式的，返回一个二元组，二元组的第2项设置成固定的P值0.5
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
SelectKBest(lambda X,Y:array(map(lambda x:mic(x,Y),X.T)),k=2).fit_transform(iris.data)

## Wrapper
### 递归特征消除法
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
RFE(estimator=LogisticRegression(),n_features_to_select=2).fit_transform(iris.data)

## Embedded
### 基于惩罚项的特征选择
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
SelectFromModel(LogisticRegression(penalty='l1',C=0.1)).fit_transform(iris.data,iris.target)


实际上，L1惩罚项降维的原理在于保留多个对目标值具有同等相关性的特征中的一个，所以没选到的特征不代表不重要。故，可结合L2惩罚项来优化。具体操作为：若一个特征在L1中的权值为1，选择在L2中权值差别不大且在L1中权值为0的特征构成同类集合，将这一集合中的特征平分L1中的权值，故需要构建一个新的逻辑回归模型：

In [ ]:
class LR(LogisticRegression):
    def __init__(self,threshold=0.01,dual=False,tol=1e-4,C=1.0,fit_intercept=True,intercept_scaling=1,class_weight=None,
                    random_state=None,solver='liblinear',max_iter=100,multi_class='ovr',verbose=0,warm_start=False,n_jobs=1):
        # 权值相近的阈值
        self.threshold = threshold
        LogisticRegression.__init__(self,penalty='l1',dual=dual,tol=tol,C=C,
        fit_intercept=fit_intercept,intercept_scaling=intercept_scaling,class_weight=class_weight,
        random_state=random_state,solver=solver,max_iter=max_iter,
        multi_class=multi_class,verbose=verbose,warm_start=warm_start,n_jobs=n_jobs)
        # 使用同样的参数创建L2逻辑回归
        self.l2 = LogisticRegression(penalty='l2',dual=dual,tol=tol,,C=C,fit_intercept=fit_intercept, intercept_scaling=intercept_scaling, class_weight = class_weight, random_state=random_state, solver=solver, max_iter=max_iter, multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)

    def fit(self,X,y,sample_weight=None):
        super(LR,self).fit(X,y,sample_weight=sample_weight)
        self.coef_old_ = self.coef_.copy()
        self.l2.fit(X,y,sample_weight=sample_weight)
        cntOfRow,cntOfCol = self.coef_.shape
        #权值系数矩阵的行数对应目标值的种类数目
        for i in range(cntOfRow):
            for j in range(cntOfCol):
                coef = self.coef_[i][j]
                #L1逻辑回归的权值系数不为0
                if coef != 0:
                    idx = [j]
                    #对应在L2逻辑回归中的权值系数
                    coef1 = self.l2.coef_[i][j]
                    for k in range(cntOfCol):
                        coef2 = self.l2.coef_[i][k]
                        #在L2逻辑回归中，权值系数之差小于设定的阈值，且在L1中对应的权值为0
                        if abs(coef1-coef2) < self.threshold and j != k and self.coef_[i][k] == 0:
                            idx.append(k)
                    #计算这一类特征的权值系数均值
                    mean = coef / len(idx)
                    self.coef_[i][idx] = mean
        return self

from sklearn.feature_selection import SelectFromModel
SelectFromModel(LR(threshold=0.5,C=0.1)).fit_transform(iris.data,iris.target)

In [ ]:
### 基于树模型
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
SelectFromModel(GradientBoostingClassifier()).fit_transform(iris.data,iris.target)

In [ ]:
# 降维

## PCA
from sklearn.decomposition import PCA
#主成分分析法，返回降维后的数据
#参数n_components为主成分数目
PCA(n_components=2).fit_transform(iris.data)

## LDA
from sklearn.lda import LDA
#线性判别分析法，返回降维后的数据
#参数n_components为降维后的维数
LDA(n_components=2).fit_transform(iris.data, iris.target)